# Multiple Input and Output Channels

**Multiple Input Channels**

![Cross-correlation with 2 input channels](http://www.d2l.ai/_images/conv_multi_in.svg)

In [2]:
import sys
sys.path.append('../..')
import d2l
from mxnet import nd

def corr2d_multi_in(X, K):
    # First, traverse along the 0th dimension (channel dimension) of X and K. 
    # Then, add them together by using * 
    return nd.add_n(*[d2l.corr2d(x, k) for x, k in zip(X, K)])

/anaconda3/envs/cs231n/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [30]:
def my_corr2d_multi_in(X, K):
    res = nd.Convolution(data=X, weight=K, kernel=(K.shape[-2], K.shape[-1]), num_filter=1, no_bias=True)
    return nd.squeeze(res)

We can construct the input array `X` and the kernel array `K` of the above diagram to validate the output of the cross-correlation operation.

In [31]:
X = nd.array([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
              [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = nd.array([[[0, 1], [2, 3]], [[1, 2], [3, 4]]])

corr2d_multi_in(X, K).shape

(2, 2)

In [32]:
X_1 = nd.expand_dims(X, axis=0)
K_1 = nd.expand_dims(K, axis=0)
my_corr2d_multi_in(X_1, K_1)


[[ 56.  72.]
 [104. 120.]]
<NDArray 2x2 @cpu(0)>

**Multiple Output Channels**

For multiple output channels we simply generate multiple outputs and then stack them together. 

In [33]:
def corr2d_multi_in_out(X, K):
    # Traverse along the 0th dimension of K, and each time, perform cross-correlation 
    # operations with input X. All of the results are merged together using the stack function.
    return nd.stack(*[corr2d_multi_in(X, k) for k in K])

In [34]:
def my_corr2d_multi_in_out(X, K):
    res = nd.Convolution(data=X, weight=K, kernel=(K.shape[-2], K.shape[-1]), num_filter=K.shape[0], no_bias=True)
    return nd.squeeze(res)

We construct a convolution kernel with 3 output channels by concatenating the kernel array `K` with `K+1` (plus one for each element in `K`) and `K+2`.

In [35]:
K = nd.stack(K, K + 1, K + 2)
K.shape

(3, 2, 2, 2)

We can have multiple input and output channels.

In [5]:
print(X.shape)
print(K.shape)
print(corr2d_multi_in_out(X, K))

(2, 3, 3)
(3, 2, 2, 2)

[[[ 56.  72.]
  [104. 120.]]

 [[ 76. 100.]
  [148. 172.]]

 [[ 96. 128.]
  [192. 224.]]]
<NDArray 3x2x2 @cpu(0)>


In [37]:
X_expand = nd.expand_dims(X, axis=0)
print(my_corr2d_multi_in_out(X_expand, K))


[[[ 56.  72.]
  [104. 120.]]

 [[ 76. 100.]
  [148. 172.]]

 [[ 96. 128.]
  [192. 224.]]]
<NDArray 3x2x2 @cpu(0)>


## $1\times 1$ Convolutions

![1x1 convolutions](http://www.d2l.ai/_images/conv_1x1.svg)

In [38]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = nd.dot(K, X)  # Matrix multiplication in the fully connected layer.
    return Y.reshape((c_o, h, w))

In [51]:
def my_corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    X = X.reshape(c_i, -1)
    K = nd.squeeze(K)
    Y = nd.dot(K, X)
    return Y.reshape(K.shape[0], h, w)
    

This is equivalent to cross-correlation with an appropriately narrow $1\times 1$ kernel.

In [46]:
X = nd.random.uniform(shape=(3, 3, 3))
K = nd.random.uniform(shape=(2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)

(Y1 - Y2).norm().asscalar() < 1e-6

True

In [50]:
Y1 = my_corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)

(Y1 - Y2).norm().asscalar() < 1e-6

(2, 3)


True